In [1]:
import os, warnings
from functools import partial

import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import geometry

# utils
from joblib import Parallel, delayed
from pandas import IndexSlice as idx
from IPython.display import display

# viz
import folium

# local imports
from spatial_interpolation import data, utils
from spatial_interpolation.visualization import map_viz
from spatial_interpolation.utils import tqdm_joblib

import dotenv

# configs
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore', message='.*initial implementation of Parquet.*')
dotenv.load_dotenv()

True

### Load the data available:

In [3]:
buoy_df, buoy_gdf = data.load_buoy_data(period=("2011-01-01", "2022-01-01"))
world_countries = data.load_world_borders()
world_countries = world_countries[world_countries.overlaps(geometry.box(*buoy_gdf.total_bounds.tolist()))]
# Remove the "buoys" that appear to be in land
land_buoys = buoy_gdf.geometry.apply(lambda x: any(world_countries.intersects(x,align=False)))
buoy_gdf = buoy_gdf[~land_buoys]
buoy_df = buoy_df[buoy_df.index.get_level_values("buoy_id").isin(buoy_gdf.index.get_level_values("buoy_id").unique())]

In [30]:
pd.options.plotting.backend = "plotly"
fig = (
    buoy_df
    .loc[idx["42019","1995-01-01":],["wave_height"]]
    .loc["42019"]
    .resample("5D").mean()
    .plot(title=\
        f"Historical of Wave Height for Buoy {buoy_gdf.loc[2011,'42019'].buoy_name}",
        labels=dict(value="Wave Height (meters)",variable=""),
    )
)
fig

In [34]:
from folium import plugins
import folium

gdf = data.NDBCData(buoy_df,buoy_gdf).join(resample="1D")
new_map = map_viz.make_folium_map(gdf.dissolve().centroid.iloc[0], zoom_start=5)

map_viz.add_time_heatmap(gdf, zoom_start=4, format="%b %d, %Y")

/tmp/ipykernel_17099/3386581746.py:5: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  new_map = map_viz.make_folium_map(gdf.dissolve().centroid.iloc[0], zoom_start=5)
/export/usuarios01/ssviloria/ml-spatial-interpolation/src/spatial_interpolation/visualization/map_viz.py:189: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  mp = make_folium_map(gdf.dissolve().centroid.iloc[0],**map_args)
/export/usuarios01/ssviloria/ml-spatial-interpolation/src/spatial_interpolation/visualization/map_viz.py:196: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf["centroids"] = gdf.geometry.cen

In [31]:
# locations with the most data points
most_points = buoy_df[["wave_height","wind_direction"]].groupby(
    "buoy_id"
).count().sort_values(
    by=["wave_height","wind_direction","buoy_id"],
    ascending=False)
print("Locations with the most data points:")
display(most_points.head(10))
# times with the most data points
most_points = buoy_df[["wave_height","wind_direction"]].groupby(
    "time"
).count().sort_values(
    by=["wave_height","wind_direction","time"],
    ascending=False)
print("Times with the most data points:")
display(most_points.head(10))

Locations with the most data points:


,wave_height,wind_direction
buoy_id,,
41008,92660,92866
42012,89844,85012
41009,88162,86921
41043,86445,77620
41049,86397,81394
41044,85123,83200
41046,84803,85603
42055,84433,79456
42020,83648,82290


Times with the most data points:


,wave_height,wind_direction
time,,
2011-07-02 19:00:00,34,34
2011-07-02 18:00:00,34,34
2011-07-02 17:00:00,34,34
2011-07-02 16:00:00,34,34
2011-07-02 13:00:00,34,34
2011-07-01 18:00:00,34,34
2011-07-01 16:00:00,34,34
2011-07-01 15:00:00,34,34
2011-07-01 14:00:00,34,34


In [37]:
variable = "wave_height"
time = "2020-05-30 01:00" #buoy_df.index.get_level_values("time").max()
map_viz.add_geodf_to_map(
    buoy_df.loc[idx[:,time],:]
        .join(buoy_gdf.loc[pd.Timestamp(time).year])
        .dropna(subset=["wave_height","wind_direction"]),
    radius=1,weight=5,
    map_args={"zoom_start":4.5},
    popup=lambda x: f"{x.buoy_name} {x.name}",
    color = lambda x: "red" if x.name[0] in ["42019"] else "blue")

In [20]:
set_1_buoys = ["41036","42040","42065","41041"]
entire_area = geometry.box(*(-98.251934,12.282308,-45,35.55)) 
area_1 = geometry.box(*(-97.3,18.930645,-81.2,30.366655))
area_2 = geometry.box(*(-82.836914,25.799891,-68.115234,35.55))
box1_gdf = gpd.GeoDataFrame({"geometry":[area_1.union(area_2)]},crs="epsg:4326")
outer_box_gdf = gpd.GeoDataFrame({"geometry":[entire_area]},crs="epsg:4326")

available_buoys = buoy_df.index.get_level_values("buoy_id").unique()
available_buoys_gdf = buoy_gdf.loc[buoy_gdf.index.get_level_values("buoy_id").isin(available_buoys)].reset_index()
uniques = buoy_gdf.loc[idx[:,set_1_buoys],:].reset_index().drop_duplicates(subset="buoy_id",keep="last").set_index("buoy_id")
mp = map_viz.make_map_of_buoys(
    location = available_buoys_gdf.unary_union.centroid.coords[0][::-1],
    zoom_start = 4,
    buoy_locations_df=available_buoys_gdf, 
    marker=partial(folium.CircleMarker, radius=1, weight=5,color="black"),
)
map_viz.add_geodf_to_map(uniques.loc[set_1_buoys],map=mp, color="red", radius=1, weight=5, layer_name="buoys from eval set 1")
area_1_gdf = gpd.GeoDataFrame({"geometry":[area_1]},crs="epsg:4326")
map_viz.add_geodf_to_map(utils.flip_coords(area_1_gdf),map=mp, layer_name="box2", color="yellow", alpha=0.5)
map_viz.add_geodf_to_map(utils.flip_coords(box1_gdf),map=mp, layer_name="box1",color="red")
map_viz.add_geodf_to_map(utils.flip_coords(outer_box_gdf),map=mp, layer_name="outer box", color="blue")
folium.LayerControl().add_to(mp)
mp

In [21]:
# within_box_buoys = buoy_gdf.loc[buoy_gdf.within(box1_gdf.unary_union)].index.get_level_values("buoy_id").unique()
# buoys_df_within = buoy_df.loc[buoy_df.index.get_level_values("buoy_id").isin(within_box_buoys)]
# prop_within_box = len(buoys_df_within.loc[set_1_buoys])/len(buoys_df_within)
# prop_within_outer = len(buoy_df.loc[set_1_buoys])/len(buoy_df)
# print(f"Proportion of eval data within box 1: {prop_within_box:.3f}")
# print(f"Proportion of eval data within outer box: {prop_within_outer:.3f}")

### Run Feature Extraction Experiment(s)

In [3]:
from experiments.noaa.machine_learning.feature_extraction import FeatureExtractionExperiment
from joblib import Parallel, delayed

def run_make_features_at_year(conf, year, **kwargs):
    experiment = FeatureExtractionExperiment(conf, year=year, **kwargs)
    experiment.set_data((buoy_df, buoy_gdf))
    _ = experiment.run()
    return _

In [5]:
from IPython.utils import io

set_conf = "set4"
with io.capture_output() as captured:
    for y in range(2011,2022):
        run_make_features_at_year(set_conf, y, verbose=False)

making features at each point..:  97%|█████████▋| 3750/3860 [00:30<00:00, 135.60it/s]

All features made. Total number of features: 23. Number of points: 3858
All features made. Total number of features: 23. Number of points: 3860


making features at each point..:  98%|█████████▊| 3792/3860 [00:31<00:00, 135.60it/s]

All features made. Total number of features: 23. Number of points: 3860
All features made. Total number of features: 23. Number of points: 3858


making features at each point..:  92%|█████████▏| 3500/3798 [00:31<00:02, 110.15it/s]

...

All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260


making features at each point..: 100%|██████████| 260/260 [00:02<00:00, 114.00it/s]


All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 261
All features made. Total number of features: 23. Number of points: 260


making features at each point..: 100%|██████████| 260/260 [00:02<00:00, 106.46it/s]


All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 261
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 261
All features made. Total number of features: 23. Number of points: 261
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 261
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All features made. Total number of features: 23. Number of points: 260
All fe

In [24]:
config = FeatureExtractionExperiment("set1").get_config()
train_df = pd.concat(
    [pd.read_parquet(f"{config.output.train_dir}/{year}.parquet") for year in range(2011,2022)],
    axis=0).sort_index()
test_df = pd.concat(
    [pd.read_parquet(f"{config.output.eval_dir}/{year}.parquet") for year in range(2011,2022)],
    axis=0).sort_index()

In [27]:
locations_within_area = buoy_gdf.loc[buoy_gdf.within(config.area)].index.get_level_values("buoy_id").unique()
print("Training set:")
print("Shape:", train_df.shape)
print("All within area?",train_df.index.get_level_values("location_id").isin(locations_within_area).all())
print("Any eval buoys in train?",train_df.index.get_level_values("location_id").isin(config.locations_full).sum())
print("Any eval index in train?",train_df.index.isin(config.split_strategy.params.eval).sum())

print("Eval set:")
print("Shape:", test_df.shape)
print("All within area?",test_df.index.get_level_values("location_id").isin(locations_within_area).all())
print("Any eval buoys?",test_df.index.get_level_values("location_id").isin(config.locations_full).sum())
print("Any eval index?",test_df.index.isin(config.split_strategy.params.eval).sum())

Training set:
Shape: (926401, 23)
All within area? True
Any eval buoys in train? 0
Any eval index in train? 0
Eval set:
Shape: (66314, 23)
All within area? True
Any eval buoys? 64666
Any eval index? 66314


---